# paiza スキルチェックの成績を自動的に取得するやつ

* 木村くんのやつをベースに改変
* 実行日の取得は未実装

In [ ]:
!pip install selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

In [ ]:
import time

import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [ ]:
def login(email, password):
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome('chromedriver', options=options)
    driver.get("https://paiza.jp/sign_in")
    time.sleep(2)
    driver.find_element_by_id("email").send_keys(email)
    driver.find_element_by_id("password").send_keys(password)
    time.sleep(1)
    driver.find_element_by_class_name('a-button-primary-large').click()
    time.sleep(2)
    return driver

In [ ]:
email = "dfkgw1978@gmail.com"
password = input().strip()
driver = login(email, password)

In [ ]:
def get_result(driver):
    columns = ["受験状態", "Title", "言語", "タイム", "バイト", "点数"]
    df = pd.DataFrame([], columns=columns)
    for rank in "dcbas":
        url_tmp = f'https://paiza.jp/challenges/ranks/{rank}/info'
        driver.get(url_tmp)
        time.sleep(2)

        problem_ids = []
        for elem in driver.find_elements_by_class_name("problem-box--submitted"):
            problem_id = elem.get_attribute("id")
            if problem_id.startswith("challenge_"):
                problem_id = int(problem_id[10:])
                problem_ids.append(problem_id)

        for problem_id in problem_ids:
                print(rank, problem_id)
                url_tmp = f'https://paiza.jp/career/challenges/{problem_id}/page/result'
                driver.get(url_tmp)
                WebDriverWait(driver, 5).until(
                    EC.presence_of_element_located((By.CLASS_NAME, "summary-box__result"))
                )
                tmp = driver.find_elements_by_css_selector(".strong.font16")
                tmp = [x.text for x in tmp]
                title = driver.find_element_by_css_selector(".summary-box__problem-name").text
                tmp = ["済", title, *tmp]
                df = df.append(pd.Series(tmp, index=df.columns), ignore_index=True)
                #break
        #break
    return df

In [ ]:
df = get_result(driver)

d 44
d 421
d 395
d 352
d 348
d 345
d 332
d 328
d 322
d 319
d 317
d 313
d 307
d 300
d 293
d 277
d 275
d 273
d 271
d 262
d 260
d 258
d 249
d 243
d 222
d 209
d 188
d 149
d 140
d 130
d 114
d 110
d 105
d 101
d 99
d 72
d 70
d 67
d 64
d 59
d 58
d 53
d 40
d 36
d 34
d 31
c 359
c 337
c 351
c 340
c 318
c 287
c 296
c 291
c 266
c 261
c 256
c 239
c 231
c 196
c 181
c 171
c 121
c 119
c 98
c 91
c 85
c 73
c 68
c 60
c 54
c 51
c 18
c 16
b 350
b 353
b 167
b 77
b 74
b 63
b 69
b 61
b 57
b 15
a 349
a 272
a 202
s 344
s 310
s 208
s 183
s 65


In [ ]:
path = "hogehoge.csv"
df.to_csv(path, encoding="utf_8_sig")